## Dataset Path configuration

<pre>
Please modify <b>config_path.py</b> first.

Raw datasets will be organized as the following structure:

dataset_path/
    | kitti/               # KITTI object detection 3D dataset
        | training/
        | testing/
    | argo/                # Argoverse dataset v1.1
        | train1/
        | train2/
        | train3/
        | train4/
        | val/
        | test/
    | nusc/                # nuScenes dataset v1.0
        | maps/
        | samples/
        | sweeps/
        | v1.0-trainval/
    | lyft/                # Lyft Level 5 dataset v1.02
        | v1.02-train/
    | waymo/               # Waymo dataset v1.0
        | training/
        | validation/
</pre>

In [ ]:
import sys
sys.path.insert(0, "..")
from config_path import raw_path_dic as path_dic

---
## Download the datasets
We provide scripts for automatic downloading. 

If you already have some of these datasets, you may create soft links following our path configuration and skip the corresponding download scripts.

### KITTI

In [ ]:
from download import download_kitti

download_kitti(path_dic["kitti"])

### Argoverse

In [ ]:
from download import download_argo

download_argo(path_dic["argo"])

### nuScenes
Please visit [this link](https://www.nuscenes.org/download)

### Lyft
Please visit [this link](https://level5.lyft.com/dataset/download-dataset/)

### Waymo

In [ ]:
from download import download_waymo

download_waymo(path_dic["waymo"])

---

## Convert the datasets to KITTI format
We use KITTI format as the uniform interface for data loading, and convert all other datasets to KITTI format.

### Argoverse

In [ ]:
!pip install git+git://github.com/argoai/argoverse-api.git

^C


  Cloning git://github.com/argoai/argoverse-api.git to c:\users\hp\appdata\local\temp\pip-req-build-j1_uj_o5


  Running command git clone --filter=blob:none --quiet git://github.com/argoai/argoverse-api.git 'C:\Users\hp\AppData\Local\Temp\pip-req-build-j1_uj_o5'
  fatal: Could not read from remote repository.

  Please make sure you have the correct access rights
  and the repository exists.
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet git://github.com/argoai/argoverse-api.git 'C:\Users\hp\AppData\Local\Temp\pip-req-build-j1_uj_o5' did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet git://github.com/argoai/argoverse-api.git 'C:\Users\hp\AppData\Local\Temp\pip-req-build-j1_uj_o5' did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [4]:
from argo2kitti import argo_to_kitti

ModuleNotFoundError: No module named 'argoverse'

### nuScenes

In [ ]:
!pip install git+git://github.com/nutonomy/nuscenes-devkit.git
from convert import nusc_to_kitti

nusc_to_kitti(path_dic["nusc"], path_dic["nusc-in-kitti-format"])

### Lyft

In [ ]:
!pip install -U git+https://github.com/lyft/nuscenes-devkit
from convert import lyft_to_kitti

lyft_to_kitti(path_dic["lyft"], path_dic["lyft-in-kitti-format"])

### Waymo

In [ ]:
!pip install tensorflow==1.15      # CPU
#!pip install tensorflow-gpu==1.15  # GPU
!pip install git+git://github.com/waymo-research/waymo-open-dataset.git
from convert import waymo_to_kitti

waymo_to_kitti(path_dic["waymo"], path_dic["waymo-in-kitti-format"])

---

## Subsample & Re-split
We subsampled and re-split the datasets to balance training time and enable offline evaluation.

We provide our split lists for result reproduction.

In [ ]:
from split import replace_split

replace_split(path_dic)

In [ ]:
import numpy as np
from scipy.integrate import quad

def entanglement_entropy(m, kappa, lambda_):
    lambda_plus = (kappa * (1 - lambda_) + lambda_ * (1 - kappa))**2
    lambda_minus = (kappa * (1 - lambda_) - lambda_ * (1 - kappa))**2
    
    def f(x):
        term1 = (1 / (2 * np.pi * lambda_)) * ((lambda_plus - x) * (x - lambda_minus)) / (x * (1 - x))
        term2 = (1 - kappa / lambda_) if lambda_ > kappa else 0
        return term1 + term2
    
    def integrand(x):
        return -x * np.log(x) - (1 - x) * np.log(1 - x) * f(x)
    
    integral, _ = quad(integrand, lambda_minus, lambda_plus)
    return m * integral

N = 1000
# To compute the entanglement entropy for specific values of kappa and lambda, you can call:
result = entanglement_entropy(N, kappa, lambda_)